In [1]:
import keras
import numpy as np
import os
import random
from sklearn.preprocessing import LabelEncoder
from  keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from keras.models import Sequential

C:\Users\Jakub\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
D=[]
check=[]
label = []
for folder in ["ok","nok"]:
    for file in os.listdir(folder):
        if(".npy" in file):
            spectogram = np.load(folder+"/"+file)
            if spectogram.shape != (128, 128): continue
            D.append((spectogram, folder))
            label.append(folder)
            check.append(spectogram)
print(len(D),len(D[0]),"done")

525 2 done


In [3]:
print(np.amin(keras.utils.normalize(check,axis=-1,order=2)),np.amax(keras.utils.normalize(check,axis=0,order=2)))

3.5831650851143464e-05 0.9496048474364612


In [4]:
dataset_array = keras.utils.normalize(check,axis=-1,order=2)
dataset = [(check[x],label[x]) for x in range(len(check))]

print(len(D))
print(len(D[0]))
print(len(D[0][0]))
random.shuffle(dataset)

train = dataset[:475]
test = dataset[475:]

x_train, y_train = zip(*train)
x_test, y_test = zip(*test)

# Reshape for CNN input
x_train = np.array([x.reshape( (128, 128, 1) ) for x in x_train])
x_test = np.array([x.reshape( (128, 128, 1) ) for x in x_test])

# One-Hot encoding for classes
encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)
print(y_test)

525
2
128
[1 0 0 1 0 1 1 1 0 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 0 0
 1 1 0 1 0 0 0 0 1 1 1 1 1]


In [5]:
Tensor_board = keras.callbacks.TensorBoard(log_dir='./logs')

In [6]:
model = Sequential()
input_shape=(128, 128, 1)

model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(1, activation='sigmoid'))

In [8]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
          epochs=50,
          batch_size=128,
          callbacks = [Tensor_board])

score = model.evaluate(x_test, y_test, batch_size=128)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
## tensorboard --logdir C:\Users\Jakub\praca_inzynierska\Scissors_falling\logs --port=8080

Epoch 1/50
475/475 [==============================] - 25s 53ms/step - loss: 0.6638 - acc: 0.6505
Epoch 2/50
475/475 [==============================] - 22s 46ms/step - loss: 0.6505 - acc: 0.6505
Epoch 3/50
475/475 [==============================] - 22s 47ms/step - loss: 0.6533 - acc: 0.6505
Epoch 4/50
475/475 [==============================] - 22s 47ms/step - loss: 0.6525 - acc: 0.6505
Epoch 5/50
475/475 [==============================] - 22s 47ms/step - loss: 0.6456 - acc: 0.6505
Epoch 6/50
475/475 [==============================] - 23s 48ms/step - loss: 0.6533 - acc: 0.6505
Epoch 7/50
475/475 [==============================] - 23s 48ms/step - loss: 0.6483 - acc: 0.6505
Epoch 8/50
475/475 [==============================] - 22s 46ms/step - loss: 0.6463 - acc: 0.6505
Epoch 9/50
475/475 [==============================] - 23s 47ms/step - loss: 0.6546 - acc: 0.6505
Epoch 10/50
384/475 [=======================>......] - ETA: 4s - loss: 0.6530 - acc: 0.6536 

KeyboardInterrupt: 